In [4]:
import os
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import re
from db import call_ignition, call_db_json, insert_many_with_df, update_ignition
from queries import hk_flag, get_set_flag
from util_func import get_mondays
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import bisect
import math

throttle_2880_2841.pth


In [65]:
kerf = 10


class KanbanCash:
    def __init__(self):
        self.kanban_respond = None
        self.df = None
        self.mondays = None
        self.mondays_str = None
        self.stock = None
        self.demand = None
        self.procs = None
        self.processing_priority = None
        self.release_df = None

    def set_kanban(self, df, mondays, mondays_str):
        self.df = df
        self.mondays_str = mondays_str
        self.mondays = mondays
        self.calc_kanban_processing_priority()

    def set_stock(self, input):
        self.stock = input

    def set_demand(self, input):
        self.demand = input

    def set_procs(self, input):
        self.empty = False
        self.procs = input

    def update_buff(self, target, buff):
        self.df.loc[self.df["target"] == target, "buff"] = buff

    def calc_kanban_processing_priority(self):
        facilities = [
            "GY3",
            "B0090",
            "B0080",
            "B0070",
            "B0060",
            "B0050",
            "B0040",
            "B0025",
            "B0021",
            "B0030",
            "B0020",
            "B0012",
        ]
        processing_priority = pd.DataFrame(
            columns=["item_description", "facility", "gy_due"]
        )
        self.df["bpoint"] = ""
        for row in self.df.itertuples():
            r = row._asdict()
            temp_df = pd.DataFrame(columns=["facility", "qty", "gy_due"])
            for facility in facilities:
                if facility == "GY3" or r[facility] != 0:
                    temp_df = temp_df.append(
                        {
                            "item_description": row.target,
                            "facility": facility,
                            "qty": r[facility],
                            "gy_due": "",
                        },
                        ignore_index=True,
                    )
                else:
                    continue
            week_index = 31
            stock = 0
            gy_procs = r["processing"] + r["GY3"]

            for i, v in enumerate(row[week_index:47]):
                gy_procs -= v
                if gy_procs < 0:
                    self.df.loc[
                        self.df["target"] == row.target, "bpoint"
                    ] = f"{self.mondays_str[week_index + i - 31]}/{str(int(v + gy_procs))}/{str(int(-gy_procs))}"
                    break
            for r2 in temp_df.itertuples():
                stock += r2.qty
                for i, v in enumerate(row[week_index:48]):
                    if v == 0:
                        continue
                    stock -= v
                    if stock < 0:
                        week_index += i
                        temp_df.loc[r2[0], "gy_due"] = self.mondays[week_index - 31]
                        break
            temp_df["gy_due"] = temp_df["gy_due"].shift(periods=1, fill_value="")
            temp_df = temp_df.drop(columns=["qty"])
            processing_priority = pd.concat([processing_priority, temp_df])
            processing_priority = processing_priority[
                processing_priority["gy_due"] != ""
            ]
        return processing_priority

    def calc_kanban_release(self, target_week):
        df = self.df
        days = self.mondays_str
        sql = "SELECT target, buff, pack_size FROM initial_release.dx_kanban"
        buff = pd.DataFrame(call_ignition(sql))
        release_cols = [
            "target",
            "gy1",
            "INTRAN",
            "GY1",
            "GY3",
            "GYHAAS",
            "processing",
            "length",
            "type",            
        ]
        to_add_cols = ["ex_date", "w16_demand", "target_demand", "est_stock", "alt"]
        release_df = df[release_cols].copy()
        release_df = release_df.reindex(columns=release_cols + to_add_cols)
        release_df = release_df.merge(buff, on=["target"], how="left")
        # release_df["ex_date"] = ""
        target_cols = ["target_demand", "w16_demand", "est_stock"]
        week_index = 31
        for row in df.itertuples():
            ex_stock = row.GY3 + row.GYHAAS + row.processing
            target_demand = sum(row[week_index : week_index + target_week])
            w16_demand = sum(row[week_index:47])
            est_stock = ex_stock - target_demand

            release_df.loc[release_df["target"] == row.target, target_cols] = (
                target_demand,
                w16_demand,
                est_stock,
            )
            for i, v in enumerate(row[week_index:47]):
                if v == 0:
                    continue
                ex_stock -= v
                if ex_stock < 0:
                    release_df.loc[
                        release_df["target"] == row.target, "ex_date"
                    ] = days[i]
                    break

        def cal_release(row):
            if row.est_stock > row.buff:
                return 0
            else:
                return row.buff - row.est_stock

        release_df["release_qty"] = release_df.apply(cal_release, axis=1)
        release_df = (
            release_df[release_df["ex_date"].notna()]
            .sort_values(["ex_date", "release_qty"], ascending=[True, False])
            .fillna(0)
        )
        alt_checker_df = release_df[
            (
                (release_df["release_qty"] > release_df["GY1"])
                | (release_df["release_qty"] < 10)
            )
            & (release_df["length"] >= 4000)
        ]
        for row in alt_checker_df.itertuples():
            pool = df[
                (df["target"] != row.target)
                & (df["type"] == row.type)
                & (df["GY3"] >= row.release_qty)
                & (df["length"] >= row.length)
            ].sort_values("length")
            if len(pool):
                release_df.loc[row[0], "alt"] = pool["target"].values[0]
        
        def check_release(row):
            flag = ''
            release_qty = 0
            if row.alt:
                flag = "ALTERNATIVE"            
            elif row.GY1 < row.release_qty:
                flag = "OUT_OF_STOCK"
            elif row.est_stock < 0:
                flag = "RELEASE"
                release_qty = row.pack_size if row.pack_size else row.release_qty
            elif row.pack_size > 0 and ((row.w16_demand - row.target_demand) - row.est_stock > row.pack_size * 0.5) and row.target in major:
                flag = "STACK_UP_RELEASE"            
                release_qty = row.pack_size if row.pack_size else row.release_qty            
            else :
                flag = "FAR_DEMAND"
            return pd.Series([flag, release_qty])        
        release_df[["release", 'release_qty']] = release_df.apply(check_release, axis=1)
        self.release_df = release_df
        return release_df.to_json(orient="records")


kanban = KanbanCash()


def length_arrange(length):
    if 3000 < length < 3240:
        return 3000
    elif 4000 <= length < 4300:
        return 4000
    elif 5000 <= length < 5300:
        return 5000
    else:
        return length


# mondays = get_mondays(15)


def get_g2(length, g1, pitch):
    return (length - g1) % pitch


def get_possible_quantity(long_len, long_g1, short_len, short_g1, pitch):
    possible_quantity = 0
    while (
        (
            (short_g1 + kerf <= long_g1)
            & ((long_len - long_g1) >= (short_len - short_g1))
        )
        | ((long_len - long_g1) >= short_len)
        | ((short_g1 == long_g1) & (long_len >= short_len))
    ):
        cut = 1
        front_loss = 0
        if long_g1 == short_g1:
            cut = 0
        elif short_g1 < long_g1:
            front_loss = long_g1 - short_g1
        else:
            front_loss = long_g1 + (pitch - short_g1)

        short_g2 = get_g2(short_len, short_g1, pitch)
        long_g1 = pitch - (short_g2 + kerf)
        possible_quantity += 1
        long_len -= short_len + front_loss + kerf

    return possible_quantity


def get_possible_quantity_blank(long_len, short_len):
    possible_quantity = 0
    len_at_point = [long_len]
    while long_len >= short_len:
        possible_quantity += 1
        long_len -= short_len
        if long_len != short_len:
            long_len -= kerf
        len_at_point.append(long_len)
    return possible_quantity


def rail_kanban_stock():
    sql = f"""
        SELECT concat(product_family, product_model) as type,item_number, item_description, product_length, quantity, virtual_location as gy, warehouse_location, product_g1 as g1, product_pitch as pitch, lot_number as batch_number FROM current_wip_inventory
            WHERE virtual_location in ('GY1', 'GY3', 'etc')
                AND rail = true
                AND item_description not like '%MML%'
                AND item_description not like '%STRAIGHTENED%'
            AND item_description not like '%GROUND%'
                AND product_length IS NOT NULL
                AND product_family IS NOT NULL
                AND product_model IS NOT NULL
                AND product_model != '55'||'65'
                AND quantity != 0  
        """
    stock = pd.DataFrame(call_db_json(sql))
    # stock_cash.set_data(stock.copy())
    stock.loc[stock["warehouse_location"] == "INTRAN", "gy"] = "INTRAN"
    stock.loc[stock["warehouse_location"] == "GYHAAS", "gy"] = "GYHAAS"
    stock = stock[stock["gy"] != "etc"]
    stock = stock.fillna(0)
    stockg = stock.groupby(
        [
            "gy",
            "item_description",
            "item_number",
            "type",
            "product_length",
            "g1",
            "pitch",
        ]
    )
    stockg = stockg["quantity"].sum().reset_index()
    stockg[["product_length", "quantity", "g1", "pitch"]] = stockg[
        ["product_length", "quantity", "g1", "pitch"]
    ].astype(int)
    stockg["product_length"] = stockg["product_length"].apply(length_arrange)
    stockg["is_k"] = np.where(
        stockg["item_description"].str.contains("NK"), True, False
    )
    stockg["is_k"] = stockg["item_description"].apply(
        lambda x: True if re.search(r"L.{0,1}Y", x) else False
    )
    return stockg, stock


def check_date_group(dt, mondays):
    index = bisect.bisect_right(mondays, dt.date())
    if index == len(mondays):
        return len(mondays)
        # return 18
    else:
        return index


def rail_kanban_demand(days, mondays):
    demand_sql = f"""
    SELECT CAST(product_length as INT) as length, product_family as family, product_model as model, item_description as des, CAST(order_quantity as INT) as qty, manufacturing_orders.order_number as mo, reference_number as hk, order_scheduled_due as due, printed_due, product_g1 as g1, product_pitch as pitch
        FROM manufacturing_orders
        INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
        WHERE manufacturing_orders.order_status ='10'
            AND manufacturing_orders.order_release_code = 5
            AND product_family not in ('HDR')
            AND facility_id = 'C0010'
            AND printed_due <= CURRENT_DATE + integer '{days}'
            AND product_length IS NOT NULL
            AND product_model IS NOT NULL
            AND product_model NOT in ('55', '65')
            AND reference_number like 'HK%'
            AND position('RAIL'in item_description) > 0
    """
    haas_sql = f"""
    SELECT
        CAST(o.product_length as INT) as length, o.product_family as family, product_model as model, item_description as des, CAST(order_quantity as INT) as qty, o.order_number as mo, reference_number as hk, order_scheduled_due as due, printed_due, product_g1 as g1, product_pitch as pitch,
        (
            SELECT
                string_agg(facility_id, ' ')
            FROM manufacturing_order_processes
            WHERE manufacturing_order_processes.order_number = o.order_number
        ) facilities
    FROM
        manufacturing_orders o
        WHERE o.order_release_code = 5
            AND o.order_status = '10'
            AND o.item_description LIKE '%+%'
            AND o.product_family = 'HSR'
            AND o.product_model IN ('45', '35')
            AND o.printed_due <= CURRENT_DATE + integer '{days}'
            AND o.reference_number LIKE 'HK%';
        """

    demand = pd.DataFrame(call_db_json(demand_sql))
    demand["set_only"] = False
    haas_demand = pd.DataFrame(call_db_json(haas_sql))
    haas_demand = haas_demand[haas_demand["facilities"] == "K0020"].drop(
        columns=["facilities"]
    )
    haas_demand["set_only"] = True
    demand = pd.concat([demand, haas_demand])
    demand["type"] = demand["family"] + demand["model"]
    # print(demand[demand["due"].isna()])
    demand.loc[demand["due"].isna(), "due"] = demand.loc[
        demand["due"].isna(), "printed_due"
    ]
    demand["due"] = pd.to_datetime(demand["due"])
    demand["week"] = demand["due"].apply(check_date_group, args=[mondays])
    demand["due"].fillna(date.today())
    demand["due"] = demand["due"].apply(lambda x: x.strftime("%m-%d"))
    demand["is_k"] = demand["des"].apply(
        lambda x: True if re.search(r"L.{0,1}K", x) else False
    )
    demand["is_m"] = demand["des"].apply(
        lambda x: True if re.search(r"L.{0,1}M", x) else False
    )
    demand["is_y"] = demand["des"].apply(
        lambda x: True if re.search(r"L.{0,1}Y", x) else False
    )
    demand = demand[~demand["is_m"]]
    demand["is_t"] = demand["des"].apply(
        lambda x: True if re.search(r"L.{0,2}T", x) else False
    )
    return demand


def rail_kanban_procs():
    processing_sql = f"""
    SELECT
        string_agg(order_number, ' ') as mos,
        concat(product_family, product_model) as type,
        item_description as des,
        CAST(product_length as INT) as length,
        SUM(CAST(order_quantity as INT)) as qty,
        item_number,
        (SELECT
            facility_id
        FROM manufacturing_order_processes
        WHERE operation_status != '40'
            AND manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ORDER BY operation_sequence
        LIMIT 1) facility
        FROM manufacturing_orders
        WHERE manufacturing_orders.order_status = '40'
            AND manufacturing_orders.order_release_code = 5
            AND reference_number = 'GY3'
            AND product_length IS NOT NULL
            AND product_model IS NOT NULL
            AND product_block_count = 0
            AND product_family != 'TS'
        GROUP BY (type, des, length, item_number, facility)
    """
    processing = pd.DataFrame(call_db_json(processing_sql))
    # procs_cash.set_data(processing.copy())
    processing["is_k"] = processing["des"].apply(lambda x: True if "NK" in x else False)
    processing["is_y"] = processing["des"].apply(
        lambda x: True if re.search(r"L.{0,1}Y", x) else False
    )
    return processing


def check_3(row):
    if row.target != "HSR45-3195L HALF RAIL 3" and 3000 <= row.length <= 3240:
        return True
    else:
        return False


def match_rail(lst, K):
    lst = np.asarray(lst)
    idx = np.where((lst - K) >= 0)[0][0]
    return lst[idx]


haas = [
    "HSR35-1000L(GP) RAIL",
    "HSR35-1320L(GP) RAIL",
    "HSR45-1200L(GP) RAIL",
    "HSR45-1270L(GP) RAIL",
    "HSR45-1568L(GP) RAIL",
    "HSR45-1778L(GP) RAIL",
    "HSR45-1980L(GP) RAIL",
    "HSR45-2459L(GP) RAIL",
    "HSR45-3195L(GP) RAIL",
]


def rail_map_set_color(row):
    if row.length >= 4000:
        return "yellow"
    elif (row.gy_haas in haas) or row.target == "SHS30-1216LTS-II HALF RAIL 3":
        return "blue"
    elif (
        row.target != "SR15-3240LY HALF RAIL 3"
        and (3000 <= row.length <= 3240)
        and "BLANK" not in row.target
    ):
        return "green"


facilities = [
    "B0012",
    "B0020",
    "B0021",
    "B0025",
    "B0030",
    "B0040",
    "B0060",
    "B0050",
    "B0070",
    "B0080",
    "B0090",
]


def get_rail_map(mondays_str):
    rail_map = pd.read_csv("./data_storage/rail_map.csv")
    sql = """
        SELECT * FROM initial_release.dx_kanban        
    """
    buff_color = pd.DataFrame(call_ignition(sql))
    rail_map = rail_map.merge(buff_color, on=["target"], how="left")
    rail_map["is_k"] = np.where(rail_map["target"].str.contains("BLANK"), True, False)
    rail_map["is_y"] = np.where(rail_map["target"].str.contains("LY"), True, False)
    rail_map["is_3"] = rail_map.apply(check_3, axis=1)
    new_cols = (
        list(rail_map.columns)
        + ["INTRAN", "GY1", "GY3", "GYHAAS", "g1", "pitch"]
        + facilities
        + mondays_str
    )
    rail_map = rail_map.reindex(columns=new_cols)
    rail_map = rail_map.fillna(int(0))
    rail_map["processing"] = 0
    return rail_map


def get_kanban_data(data_type, target, option):
    if data_type == "DEMAND":
        df = kanban.demand
        return df[target][option]
    elif data_type == "STOCK":
        df = kanban.stock
        if option in ["GY1", "INTRAN"]:
            target = target.replace(" BLANK", "")
        res = df[(df["target"] == target) & (df["gy"] == option)].sort_values(
            "batch_number"
        )
        return res.to_dict(orient="records")
    else:
        df = kanban.procs
        res = df[(df["target"] == target) & (df["facility"] == option)]
        return res.to_dict(orient="records")


def update_rail_kanban(weeks=15):
    # mondays = get_mondays(15)
    mondays_time = get_mondays(15)
    mondays = [i.date() for i in mondays_time]
    mondays_str = [d.strftime("%m-%d") for d in mondays]
    stock, stock_temp = rail_kanban_stock()
    demand = rail_kanban_demand(weeks * 7, mondays)
    procs = rail_kanban_procs()
    rail_map = get_rail_map(mondays_str)
    rail_types = pd.unique(rail_map["type"])
    demand_map = {}
    for r_type in rail_types:
        local_stock = stock[stock["type"] == r_type]
        local_map = rail_map[rail_map["type"] == r_type]
        local_demand = demand[demand["type"] == r_type]
        local_procs = procs[procs["type"] == r_type]
        length_pool = pd.unique(local_map["length"])
        for row in local_map.itertuples():
            GY3_stock_list = [row.target, row.sub_target]
            GY3_stock = local_stock[
                local_stock["item_description"].isin(GY3_stock_list)
            ]["quantity"].sum()
            try:
                GY1_stock_list = [row.gy1_hardened, row.gy1, row.gy1_sub]
            except:
                print(row)
            if row.is_k == False:
                g1 = local_stock[local_stock["item_description"].isin(GY3_stock_list)][
                    "g1"
                ].values
                GY1_stock_list = [row.gy1_hardened, row.gy1, row.gy1_sub]
                GY1_stock_items = local_stock[
                    local_stock["item_description"].isin(GY1_stock_list)
                ]
                in_transit = GY1_stock_items[GY1_stock_items["gy"] == "INTRAN"][
                    "quantity"
                ].values
                GY1_stock = GY1_stock_items[GY1_stock_items["gy"] != "INTRAN"][
                    "quantity"
                ].sum()
                rail_map.loc[row[0], "g1"] = g1[0] if len(g1) else 0
            temp_proc = local_procs[local_procs["des"].isin(GY3_stock_list)][
                ["facility", "qty"]
            ].values
            procs.loc[procs["des"].isin(GY3_stock_list), "target"] = row.target
            proc_sum = 0
            for facility, qty in temp_proc:
                rail_map.loc[row[0], facility] = qty
                proc_sum += qty
            haas_qty = local_stock[(local_stock["item_description"] == row.gy_haas)][
                "quantity"
            ].values
            stock.loc[stock["item_description"] == row.gy_haas, "target"] = row.target
            procs.loc[procs["des"].isin(GY3_stock_list), "target"] = row.target
            stock_list = GY3_stock_list + GY1_stock_list
            stock.loc[stock["item_description"].isin(stock_list), "target"] = row.target
            stock_temp.loc[
                stock_temp["item_description"].isin(stock_list), "target"
            ] = row.target
            rail_map.loc[row[0], "GYHAAS"] = haas_qty[0] if len(haas_qty) else 0
            rail_map.loc[row[0], "processing"] = proc_sum
            rail_map.loc[row[0], "GY3"] = GY3_stock
            rail_map.loc[row[0], "GY1"] = GY1_stock
            rail_map.loc[row[0], "INTRAN"] = in_transit[0] if len(in_transit) else 0
        for r in local_demand.itertuples():
            k_flag = r.is_k
            y_flag = r.is_y
            if (r.des in haas) or r.set_only:
                source_length = r.length
                if r.length == 1000:
                    source_qty = r.qty // 2
                else:
                    source_qty = r.qty
                source = local_map[local_map["length"] == source_length]
            else:
                if r.is_t:
                    source = local_map[
                        local_map["is_3"] & (local_map["is_k"] == k_flag)
                    ]

                    qty_for_3m = math.ceil(r.length / 3000)
                    source_qty = r.qty * qty_for_3m
                else:
                    source_length = match_rail(length_pool, r.length)
                    if row.type == "SR25":
                        source = local_map[
                            (local_map["is_k"] == k_flag)
                            & (local_map["length"] == source_length)
                        ]
                    else:
                        source = local_map[
                            (local_map["is_k"] == k_flag)
                            & (local_map["is_y"] == y_flag)
                            & (local_map["length"] == source_length)
                        ]

                    if k_flag:
                        qty_in_source = get_possible_quantity_blank(
                            source_length, r.length
                        )
                    else:
                        temp = source["g1"].values
                        if len(temp):
                            source_g1 = source["g1"].values[0]
                        else:
                            source_g1 = 20
                        qty_in_source = get_possible_quantity(
                            source_length, source_g1, r.length, r.g1, r.pitch
                        )
                    source_qty = math.ceil(r.qty / qty_in_source)
            source_index = source.index[0]
            target = source["target"].values[0]

            try:
                week_str = mondays_str[r.week - 1]
            except:
                print(mondays_str)
                print(r.week - 1)
            if target not in demand_map:
                demand_map[target] = {}
            if week_str not in demand_map[target]:
                demand_map[target][week_str] = []
            temp_row = r._asdict()
            temp_row["src_qty"] = source_qty
            demand_map[target][week_str].append(temp_row)
            rail_map.loc[source_index, week_str] += source_qty
    kanban.set_procs(procs)
    kanban.set_stock(stock_temp)
    kanban.set_demand(demand_map)
    kanban.set_kanban(rail_map, mondays_time, mondays_str)


def update_gy3_buffer(target, buff):
    sql = f"""
            UPDATE initial_release.dx_kanban SET buff = {buff} WHERE target = '{target}'            
        """
    kanban.update_buff(target, buff)
    return update_ignition(sql)


In [66]:
update_rail_kanban()

In [68]:
kanban.calc_kanban_release(8)
kanban.release_df


,target,gy1,INTRAN,GY1,GY3,GYHAAS,processing,length,type,ex_date,w16_demand,target_demand,est_stock,alt,buff,pack_size,release_qty,release
126,SHW27-3040L HALF RAIL 3,SHW27-3000L DRAWN RAIL,260.0,0.0,2.0,0.0,0,3040,SHW27,10-02,231.0,130.0,-128.0,0.0,0.0,0.0,0,OUT_OF_STOCK
45,HSR45-5050L HALF RAIL 3,HSR45Z-5050L DRAWN RAIL,0.0,480.0,0.0,0.0,0,5050,HSR45,10-09,6.0,6.0,-6.0,HSR45-5050L HALF RAIL 3 BLANK,0.0,0.0,0,ALTERNATIVE
83,SHS45-5050L HALF RAIL 3,SHS45-5050L DRAWN RAIL,0.0,0.0,2.0,0.0,0,5050,SHS45,10-09,3.0,3.0,-1.0,0.0,0.0,0.0,0,OUT_OF_STOCK
79,SHS45-3020L HALF RAIL 3,SHS45-3020L DRAWN RAIL,540.0,0.0,26.0,0.0,0,3020,SHS45,10-16,94.0,71.0,-45.0,0.0,0.0,0.0,0,OUT_OF_STOCK
49,SHS15-3040L HALF RAIL 3,SHS15-3040L DRAWN RAIL,720.0,0.0,387.0,0.0,240,3040,SHS15,11-13,1073.0,533.0,94.0,0.0,200.0,240.0,0,OUT_OF_STOCK
41,HSR45-2459L HALF RAIL 3,HSR45Z-5050L DRAWN RAIL,0.0,480.0,70.0,0.0,0,2459,HSR45,12-11,74.0,0.0,70.0,0.0,0.0,0.0,0,FAR_DEMAND
36,HSR45-1200L HALF RAIL 3,HSR45Z-1200L LENGTH CUT RAIL,240.0,311.0,5.0,3.0,450,1200,HSR45,12-18,482.0,0.0,458.0,0.0,0.0,30.0,0,FAR_DEMAND


In [69]:
major = [    
    "HSR15",
    "HSR20",
    "HSR25",    
    "SHS15",
    "SHS20",
    "SHS25",    
    "SR15",
    "SR20",
    "SR25"    
  ]
def calc_kanban_release(target_week):
    df = kanban.df
    days = kanban.mondays_str
    sql = "SELECT target, buff, pack_size FROM initial_release.dx_kanban"
    buff = pd.DataFrame(call_ignition(sql))
    release_cols = [
        "target",
        "gy1",
        "INTRAN",
        "GY1",
        "GY3",
        "GYHAAS",
        "processing",
        "length",
        "type",            
    ]
    to_add_cols = ["ex_date", "w16_demand", "target_demand", "est_stock", "alt"]
    release_df = df[release_cols].copy()
    release_df = release_df.reindex(columns=release_cols + to_add_cols)
    release_df = release_df.merge(buff, on=["target"], how="left")
    # release_df["ex_date"] = ""
    target_cols = ["target_demand", "w16_demand", "est_stock"]
    week_index = 31
    for row in df.itertuples():
        ex_stock = row.GY3 + row.GYHAAS + row.processing
        target_demand = sum(row[week_index : week_index + target_week])
        w16_demand = sum(row[week_index:47])
        est_stock = ex_stock - target_demand

        release_df.loc[release_df["target"] == row.target, target_cols] = (
            target_demand,
            w16_demand,
            est_stock,
        )
        for i, v in enumerate(row[week_index:47]):
            if v == 0:
                continue
            ex_stock -= v
            if ex_stock < 0:
                release_df.loc[
                    release_df["target"] == row.target, "ex_date"
                ] = days[i]
                break

    def cal_release(row):
        if row.est_stock > row.buff:
            return 0
        else:
            return row.buff - row.est_stock

    release_df["release_qty"] = release_df.apply(cal_release, axis=1)
    release_df = (
        release_df[release_df["ex_date"].notna()]
        .sort_values(["ex_date", "release_qty"], ascending=[True, False])
        .fillna(0)
    )
    alt_checker_df = release_df[
        (
            (release_df["release_qty"] > release_df["GY1"])
            | (release_df["release_qty"] < 10)
        )
        & (release_df["length"] >= 4000)
    ]
    for row in alt_checker_df.itertuples():
        pool = df[
            (df["target"] != row.target)
            & (df["type"] == row.type)
            & (df["GY3"] >= row.release_qty)
            & (df["length"] >= row.length)
        ].sort_values("length")
        if len(pool):
            release_df.loc[row[0], "alt"] = pool["target"].values[0]
    
    def check_release(row):
        flag = ''
        release_qty = 0
        if row.alt:
            flag = "ALTERNATIVE"            
        elif row.GY1 < row.release_qty:
            flag = "OUT_OF_STOCK"
        elif row.est_stock < 0:
            flag = "RELEASE"
            
            release_qty = math.ceil(row.release_qty / row.pack_size) if row.pack_size else row.release_qty
        elif row.pack_size > 0 and ((row.w16_demand - row.target_demand) - row.est_stock > row.pack_size * 0.5) and row.target in major:
            flag = "STACK_UP_RELEASE"            
            release_qty = row.pack_size
        else :
            flag = "FAR_DEMAND"
        return pd.Series([flag, release_qty])        
    release_df[["release", 'release_qty']] = release_df.apply(check_release, axis=1)
    
    return release_df

    

In [70]:
a = calc_kanban_release(8)
a


,target,gy1,INTRAN,GY1,GY3,GYHAAS,processing,length,type,ex_date,w16_demand,target_demand,est_stock,alt,buff,pack_size,release_qty,release
126,SHW27-3040L HALF RAIL 3,SHW27-3000L DRAWN RAIL,260.0,0.0,2.0,0.0,0,3040,SHW27,10-02,231.0,130.0,-128.0,0.0,0.0,0.0,0,OUT_OF_STOCK
45,HSR45-5050L HALF RAIL 3,HSR45Z-5050L DRAWN RAIL,0.0,480.0,0.0,0.0,0,5050,HSR45,10-09,6.0,6.0,-6.0,HSR45-5050L HALF RAIL 3 BLANK,0.0,0.0,0,ALTERNATIVE
83,SHS45-5050L HALF RAIL 3,SHS45-5050L DRAWN RAIL,0.0,0.0,2.0,0.0,0,5050,SHS45,10-09,3.0,3.0,-1.0,0.0,0.0,0.0,0,OUT_OF_STOCK
79,SHS45-3020L HALF RAIL 3,SHS45-3020L DRAWN RAIL,540.0,0.0,26.0,0.0,0,3020,SHS45,10-16,94.0,71.0,-45.0,0.0,0.0,0.0,0,OUT_OF_STOCK
49,SHS15-3040L HALF RAIL 3,SHS15-3040L DRAWN RAIL,720.0,0.0,387.0,0.0,240,3040,SHS15,11-13,1073.0,533.0,94.0,0.0,200.0,240.0,0,OUT_OF_STOCK
41,HSR45-2459L HALF RAIL 3,HSR45Z-5050L DRAWN RAIL,0.0,480.0,70.0,0.0,0,2459,HSR45,12-11,74.0,0.0,70.0,0.0,0.0,0.0,0,FAR_DEMAND
36,HSR45-1200L HALF RAIL 3,HSR45Z-1200L LENGTH CUT RAIL,240.0,311.0,5.0,3.0,450,1200,HSR45,12-18,482.0,0.0,458.0,0.0,0.0,30.0,0,FAR_DEMAND


In [400]:
def calc_kanban_processing_priority(df, days):        
    facilities = ['GY3', 'B0012', 'B0020', 'B0021', 'B0025', 'B0030', 'B0040', 'B0060', 'B0050', 'B0070', 'B0080', 'B0090']    
    processing_priority = []
    for row in df.itertuples():
        r = row._asdict()
        stock = 0        
        ex_stock = row.GY3 + row.GYHAAS + row.processing
        week_index = 31
        target_demand = 0
        total_demand = 0
        for facility in facilities: 
            if r[facility] == 0:
                continue
            stock += r[facility]
            for i, v in enumerate(row[week_index:48]):
                if v == 0:
                    continue                                
                stock -= v
                if stock < 0:                                        
                    processing_priority.append({"target":r['target'], "facility":facility, "gy_due":days[i]})
                    week_index += i
                    break    
    return processing_priority
a = 

In [8]:
def calc_kanban_release(self, target_week):
    df = kanban.df
    days = kanban.mondays_str
    sql = "SELECT target, buff, pack_size FROM initial_release.dx_kanban"
    buff = pd.DataFrame(call_ignition(sql))
    release_cols = [
        "target",
        "gy1",
        "INTRAN",
        "GY1",
        "GY3",
        "GYHAAS",
        "processing",
        "length",
        "type",
        "pack_size",
    ]
    to_add_cols = ["ex_date", "w16_demand", "target_demand", "est_stock", "alt"]
    release_df = df[release_cols].copy()
    release_df = release_df.reindex(columns=release_cols + to_add_cols)
    release_df = release_df.merge(buff, on=["target"], how="left")
    # release_df["ex_date"] = ""
    target_cols = ["target_demand", "w16_demand", "est_stock"]
    week_index = 31
    for row in df.itertuples():
        ex_stock = row.GY3 + row.GYHAAS + row.processing
        target_demand = sum(row[week_index : week_index + target_week])
        release_df.loc[release_df["target"] == row.target, target_cols] = (
            target_demand,
            sum(row[week_index:47]),
            ex_stock - target_demand,
        )
        for i, v in enumerate(row[week_index:48]):
            if v == 0:
                continue
            ex_stock -= v
            if ex_stock < 0:
                release_df.loc[
                    release_df["target"] == row.target, "ex_date"
                ] = days[i]
                break
calc_kanban_release(8)

TypeError: calc_kanban_release() missing 1 required positional argument: 'target_week'

In [385]:
def update_gy3_buffer(target, buff):
    sql = f"""
            UPDATE initial_release.dx_kanban SET buff = {buff} WHERE target = '{target}';
        """        
    return call_ignition(sql)


,target,gy1,GY1,GY3,GYHAAS,processing,ex_date,w16_demand,target_demand,est_stock
119,SR35-5100L HALF RAIL 3,SR35-5100L DRAWN RAIL,0.0,1.0,0.0,0,10-02,2.0,2.0,-1.0
80,SHS45-3020L HALF RAIL 3,SHS45-3020L DRAWN RAIL,0.0,4.0,0.0,60,10-09,88.0,76.0,-12.0
99,SR25-3000LY HALF RAIL 3,SR25-3040L DRAWN RAIL,677.0,1.0,0.0,92,11-06,516.0,189.0,-96.0
43,HSR45-2459L HALF RAIL 3,0,0.0,4.0,0.0,100,12-04,106.0,52.0,52.0
38,HSR45-1200L HALF RAIL 3,HSR45Z-1200L LENGTH CUT RAIL,101.0,5.0,3.0,450,12-11,482.0,244.0,214.0
51,SHS15-3040L HALF RAIL 3,SHS15-3040L DRAWN RAIL,0.0,510.0,0.0,240,12-18,761.0,510.0,240.0


In [ ]:
def get_kanban_processing_priority():
    sql = """
        select target, GY3, B0090, B0080, B0070, B0050, B0060, B0040, B0030, B0025, B0021, B0020, B0012, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16 from initial_release.dx_kanban
    """
    kanban = pd.DataFrame(call_ignition(sql))        
    days = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16']
    facilities = ['gy3',
    'b0090',
    'b0080',
    'b0070',
    'b0050',
    'b0060',
    'b0040',
    'b0030',
    'b0025',
    'b0021',
    'b0020',
    'b0012']
    res = pd.DataFrame(columns=['target', 'facility', 'gy_due'])
    for row in kanban.itertuples():
        r = row._asdict()
        stock = 0
        row_index = 14        
        for facility in facilities: 
            if r[facility] == 0:
                continue
            stock += r[facility]
            for i, v in enumerate(row[row_index:]):
                if v == 0:
                    continue
                stock -= v
                if stock < 0:
                    # try:
                    kanban.loc[kanban['target'] == r['target'], facility] = days[i]
                    # except:
                    res = res.append({"target":r['target'], "facility":facility, "gy_due":days[i]}, ignore_index=True)
                    break
    kanban.to_csv("kanban2.csv")
    return res
a =  get_kanban_processing_priority()

In [ ]:
def update_dx_kanban(rail_map, mondays_str):
    target_table = 'initial_release.dx_kanban'
    rename_dict = {}
    for index, value in enumerate(mondays_str):
        rename_dict[value] = 'w' + str(index)    
    rail_map = rail_map.rename(columns=rename_dict)
    keep_cols = ['target', 'INTRAN', 'GY1', 'GY3', 'buff', 'GYHAAS', 'B0012', 'B0020', 'B0021', 'B0025', 'B0030', 'B0040', 'B0060', 'B0050',
        'B0070', 'B0080', 'B0090', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6',
        'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'color']
    rail_map = rail_map[keep_cols]
    
    del_sql = f"""
        TRUNCATE {target_table}; DELETE FROM {target_table}
    """
    update_ignition(del_sql)
    return insert_many_with_df('ignition', rail_map, target_table)

def get_rail_kanban():
    mondays = [i.date() for i in get_mondays(15)]    
    mondays_str = [d.strftime("%m-%d") for d in mondays]
    sql = """SELECT * from initial_release.dx_kanban"""
    df = pd.DataFrame(call_ignition(sql))
    rename = {}
    cols_to_rename = ['intran', 'gy1', 'gy3', 'buff', 'gyhaas', 'b0012', 'b0020', 'b0021', 'b0025', 'b0030', 'b0040', 'b0060', 'b0050', 'b0070', 'b0080', 'b0090']
    weeks_to_rename = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16']
    for i, v in enumerate(cols_to_rename):
        rename[v] = v.upper()
    for i, v in enumerate(weeks_to_rename):
        rename[v] = mondays_str[i]
    df = df.rename(columns=rename)
    return 
    
# a.columns